<a href="https://colab.research.google.com/github/LoteeYoon/RL-practice/blob/master/DQN_CartPole_v0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [85]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import gym
import numpy as np
import matplotlib.pyplot as plt
import sys, os
from IPython.display import clear_output
from typing import List

In [86]:
class Network(nn.Module):
    def __init__(self, obs_dim: int, act_dim: int):
        super(Network, self).__init__()

        self.obs_dim = obs_dim
        self.act_dim = act_dim
        self.fc1 = nn.Linear(self.obs_dim, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, self.act_dim)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x

In [87]:
class ReplayBuffer:
    def __init__(self, memory_size: int, batch_size: int):
        self.size = 0
        self.transition_size = 5    ## transition s, a, r, s', d
        self.batch_size = batch_size
        self.max_size = memory_size
        self.replay = [] ###### numpy array로 선언하면 element type이 달라서 transition이 안 들어감
        self.sample_index = []
        self.train_batch = []
        self.ptr = 0
        

    def store(self, 
              prev_obs: List[float],
              action: float,
              reward: float,
              obs: List[float],
              done: bool,
              ):
        transition = np.array([prev_obs, action, reward, obs, done])
        ### 메모리가 처음으로 가득찰 때까지는 del 없이 채우다가 메모리 가득차면 del해주면서 처음부터 insert
        if self.ptr >= self.max_size:
            del self.replay[self.ptr % self.max_size]
            self.replay.insert(self.ptr % self.max_size, transition)
        else:
            self.replay.insert(self.ptr, transition)   ####### .insert() 제거함

        self.ptr += 1
        # self.ptr = (self.ptr + 1) % self.max_size
        self.size = min(self.size + 1, self.max_size)


    def sample_batch(self) -> List[int]:
        sample_index = np.random.randint(low=0, high=self.size, size=self.batch_size) ##### local로 만들어줌
        train_batch = np.array([self.replay[idx] for idx in sample_index]) ##### local로 만들어줌
        return train_batch
    

In [92]:
class Agent:
    def __init__(self, 
                 env: gym.Env,
                 memory_size: int,
                 batch_size: int,
                 target_update: int,
                 epsilon_decay: float,
                 max_epsilon: float = 1.0,
                 min_epsilon: float = 0.1,
                 gamma: float = 0.99):
        self.env = env
        self.batch_size = batch_size
        self.memory = ReplayBuffer(memory_size, batch_size)
        self.target_update = target_update
        self.epsilon = max_epsilon
        self.epsilon_decay = epsilon_decay
        self.max_epsilon = max_epsilon
        self.min_epsilon = min_epsilon
        self.gamma = gamma

        self.obs_dim = env.observation_space.shape[0]
        self.act_dim = env.action_space.n

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  ####### 필요성 이해하고 넣어줌

        self.dqn = Network(self.obs_dim, self.act_dim).to(self.device) # to 
        self.dqn_target = Network(self.obs_dim, self.act_dim).to(self.device)# to
        self.dqn_target.load_state_dict(self.dqn.state_dict())
        self.dqn_target.eval()

        self.optimizer = optim.Adam(self.dqn.parameters())  ####### optimizer 선언을 안 해줬었네


    def update_model(self):
        train_batch = self.memory.sample_batch()

        loss = self.compute_loss(train_batch)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        return loss.item()


    def compute_loss(self, train_batch):

        with torch.no_grad(): ######### 네 코드 참고해서 넣음. 근데 아직 no_grad 필요한 구간 스스로 판단하기 어려움
            s = torch.FloatTensor(np.array([i[0] for i in train_batch])).to(self.device)
            a = torch.LongTensor(np.array([i[1] for i in train_batch])).view([-1, 1]).to(self.device)
            r = torch.FloatTensor(np.array([i[2] for i in train_batch])).view([-1, 1]).to(self.device)
            next_s = torch.FloatTensor(np.array([i[4] for i in train_batch])).to(self.device)
            d = torch.BoolTensor(np.array([i[3] for i in train_batch])).view([-1, 1]).to(self.device)
            
            curr_value = self.dqn(s).gather(1, a)
            next_value = self.dqn_target(next_s).max(dim=1, keepdim=True)[0].detach()

            mask = 1 - done
            target = (r + self.gamma * next_value * mask).to(self.device)

        loss = F.smooth_l2_loss(curr_value, target)

        return loss


    def train(self, num_episodes: int, plot_interval: int):
        
        update_cnt = 0
        losses = []
        score = 0
        scores = []

        for iter in range(1, num_episodes + 1):

            s = self.env.reset()
            d = False
            
            while (not d):
                #self.env.render() #######

                if self.epsilon > np.random.rand(): ### rand() 로 변경 --> 0 ~ 1 사이 uniformly sample
                    a = self.env.action_space.sample()
                else:
                    a = self.dqn.forward(torch.FloatTensor(s).to(self.device)).argmax() ####### 아래 코드랑 둘 중에 뭐가 맞는 걸까?
                  # a = self.dqn(torch.FloatTensor(s).to(self.device)).argmax()
                    a = a.detach().cpu().numpy()    

                next_s, r, d, _ = self.env.step(a)
                self.memory.store(s, a, r, next_s, d)
                s = next_s
                score += r
                
                if d:
                    scores.append(score)
                    score = 0

                if self.memory.size >= self.batch_size:  ## len 제거함
                    loss = self.update_model()
                    losses.append(loss)
                    update_cnt += 1

                    self.epsilon = max(self.min_epsilon,
                                    self.epsilon - self.epsilon_decay * (self.max_epsilon - min_epsilon))

                    if update_cnt % self.target_update == 0:
                        self.dqn_target.load_state_dict(self.dqn.state_dict())

                if iter % plot_interval == 0:
                    clear_output(True)
                    plt.figure(figsize=(20, 5))
                    plt.subplot(121)
                    plt.title('iter %s. score: %s' % (iter, np.mean(scores[-10:]))) ## 가장 최근 10개 에피소드 평균 score 출력
                    plt.plot(scores)
                    plt.subplot(122)
                    plt.title('loss')
                    plt.plot(losses)
                    plt.show()

In [93]:
env = gym.make('CartPole-v0')
memory_size = 1000
batch_size = 32
target_update = 100
epsilon_decay = 1 / 2000

agent = Agent(env, memory_size, batch_size, target_update, epsilon_decay)

In [94]:
num_episodes = 100
plot_interval = 200

agent.train(num_episodes, plot_interval)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


RuntimeError: ignored

In [40]:
size = 10
tran_size = 5

replay = [[0]* tran_size for _ in range(size)]

print(replay)
print(len(replay))
print(len(replay[0]))
print(len(replay[1]))
s0 = [[1, 2], 3, 4, [5, 6], True] 
s1 = [[2, 3], 4, 5, [6, 7], False]

replay[0] = s0
replay[1] = s1

print(replay)
print(len(replay))
print(len(replay[0]))
print(len(replay[1]))

[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]
10
5
5
[[[1, 2], 3, 4, [5, 6], True], [[2, 3], 4, 5, [6, 7], False], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]
10
5
5


In [41]:
type(s1)

list

In [20]:
replay = np.array(replay)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [21]:
replay

array([[list([1, 2]), 3, 4, list([5, 6]), True],
       [list([2, 3]), 4, 5, list([6, 7]), False],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]], dtype=object)

In [22]:
s2 = [[5, 6], 7, 8, [9, 10], True]

replay[2] = s2

replay

array([[list([1, 2]), 3, 4, list([5, 6]), True],
       [list([2, 3]), 4, 5, list([6, 7]), False],
       [list([5, 6]), 7, 8, list([9, 10]), True],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]], dtype=object)

In [24]:
type(replay[0][0])

list

In [74]:
s0 = [[1, 2], 3, 4, [5, 6], True] 
s1 = [[2, 3], 4, 5, [6, 7], False]
s2 = [[5, 6], 7, 8, [9, 10], True]
s3 = [[3, 9], 3, 2, [1, 4], False]
s4 = [[1, 5], 6, 3, [10, 8], False]
replay = np.zeros([5, 5])

In [75]:
replay

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [76]:
replay[0] = s0
replay[1] = s1
replay[2] = s2
replay[3] = s3
replay[4] = s4

ValueError: ignored

In [44]:
replay

[[[1, 2], 3, 4, [5, 6], True],
 [[2, 3], 4, 5, [6, 7], False],
 [[5, 6], 7, 8, [9, 10], True],
 [[3, 9], 3, 2, [1, 4], False],
 [[1, 5], 6, 3, [10, 8], False]]

In [45]:
type(replay)

list

In [48]:
sample_index = np.random.randint(low=0, high=5, size=2)

In [49]:
sample_index

array([0, 0])

In [50]:
type(sample_index)

numpy.ndarray

In [51]:
train_batch = [replay[idx] for idx in sample_index]

In [52]:
train_batch

[[[1, 2], 3, 4, [5, 6], True], [[1, 2], 3, 4, [5, 6], True]]

In [53]:
type(train_batch)

list

In [56]:
train_batch_2 = np.array([replay[idx] for idx in sample_index])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [57]:
train_batch_2

array([[list([1, 2]), 3, 4, list([5, 6]), True],
       [list([1, 2]), 3, 4, list([5, 6]), True]], dtype=object)

In [64]:
s = torch.FloatTensor([i[0] for i in train_batch_2])

In [65]:
s

tensor([[1., 2.],
        [1., 2.]])

In [66]:
s_2 = torch.FloatTensor(np.array([i[0] for i in train_batch_2]))

In [67]:
s_2

tensor([[1., 2.],
        [1., 2.]])